# Minimal mnist example on Hopsworks
---

<font color='red'> <h3>Tested with TensorFlow 1.14</h3></font>

<p>
<h1>Machine Learning on <a href="https://github.com/logicalclocks/hopsworks">Hopsworks
</a></h1> 
</p>

![hops.png](../../images/hops.png)

## The `hops` python module

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../images/experiments.gif)

In [3]:
def minimal_mnist():
    import tensorflow as tf
    import numpy as np

    from tensorflow.examples.tutorials.mnist import input_data
    mnist = input_data.read_data_sets('MNIST_data')

    from hops import tensorboard

    def input(dataset):
        return dataset.images, dataset.labels.astype(np.int32)

    # Specify feature
    feature_columns = [tf.feature_column.numeric_column("x", shape=[28, 28])]

    # Build 2 layer DNN classifier
    classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[256, 32],
    optimizer=tf.train.AdamOptimizer(1e-4),
    n_classes=10,
    dropout=0.1,
    model_dir=tensorboard.logdir()
    )

    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.train)[0]},
    y=input(mnist.train)[1],
    num_epochs=None,
    batch_size=16,
    shuffle=True
    )

    classifier.train(input_fn=train_input_fn, steps=1000)

    # Define the test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": input(mnist.test)[0]},
    y=input(mnist.test)[1],
    num_epochs=1,
    shuffle=False
    )

    # Evaluate accuracy
    evaluate = classifier.evaluate(input_fn=test_input_fn)
    accuracy = evaluate["accuracy"]
    loss = evaluate["loss"]
    return {'accuracy': accuracy, 'loss': loss}

In [4]:
from hops import experiment
from hops import hdfs
experiment.launch(minimal_mnist,
                  name='mnist estimator', 
                  description='A minimal mnist example with two hidden layers', metric_key='accuracy')

Finished Experiment 

('hdfs://10.0.2.15:8020/Projects/demo_deep_learning_admin000/Experiments/application_1578481361664_0001_2', {'accuracy': 0.8934000134468079, 'loss': 53.07727813720703, 'log': 'Experiments/application_1578481361664_0001_2/output.log'})